In [21]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
import cirq

import optim
import torch as torch

Any computational object that can apply quantum operations and return a measurement value is called a quantum device.

In PennyLane, a device could be a hardware device (such as the IBM QX4, via the PennyLane-PQ plugin), or a software simulator (such as Strawberry Fields, via the PennyLane-SF plugin).


In [40]:
dev = qml.device('default.qubit', wires=1)

Unitary = arbitrary rotation


In [41]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
##phi = 0
##theta = 0
##omega = 0
angles = [phi,theta,omega]

def real(angles, **kwargs):
    qml.Rot(*angles, wires=0)
        

Generator

In [42]:
def generator(w, **kwargs):
    qml.RZ(w[0], wires=0)
    qml.RY(w[1], wires=0)
    qml.RZ(w[2], wires=0)

Discriminators: DI chooses the input, DO is standard


In [43]:
def DI(w):
    qml.RX(w[0], wires=0)
    qml.RY(w[1], wires=0)
    qml.RZ(w[2], wires=0)

def DO(w):
    qml.RZ(w[0], wires=0)
    qml.RY(w[1], wires=0)
    qml.RZ(w[2], wires=0)


2 nodes:  1 for data-discriminator
          1 for generator-discriminator
          

In [56]:
@qml.qnode(dev, interface="torch", diff_method = 'parameter-shift')
def real_disc_circuit(angles, disc_weights):
    #DI(disc_weights[9:18])
    real(angles)
    DO(disc_weights[0:3])
    return qml.expval(qml.PauliZ(0))

@qml.qnode(dev, interface="torch", diff_method = 'parameter-shift')
def gen_disc_circuit(gen_weights, disc_weights):
    #DI(disc_weights[9:18])
    generator(gen_weights)
    DO(disc_weights[0:3])
    return qml.expval(qml.PauliZ(0))

##grad computing node, bc the pytorch interface does strange stuff
@qml.qnode(dev, diff_method = 'parameter-shift')
def real_disc_circuit_grad(angles, disc_weights):
    #DI(disc_weights[9:18])
    real(angles)
    DO(disc_weights[0:3])
    return qml.expval(qml.PauliZ(0))

@qml.qnode(dev, diff_method = 'parameter-shift')
def gen_disc_circuit_grad(gen_weights, disc_weights):
    #DI(disc_weights[9:18])
    generator(gen_weights)
    DO(disc_weights[0:3])
    return qml.expval(qml.PauliZ(0))


Cost Functions

In [57]:
def prob_real_true(disc_weights):
    true_disc_output = real_disc_circuit(angles, disc_weights)
    
    # convert to probability
    prob_real_true = (true_disc_output + 1) / 2
    return prob_real_true


def prob_fake_true(gen_weights, disc_weights):
    
    fake_disc_output = gen_disc_circuit(gen_weights, disc_weights)
    # convert to probability
    prob_fake_true = (fake_disc_output + 1) / 2
    return prob_fake_true


def disc_cost(disc_weights, gen_weights):
    cost = prob_fake_true(gen_weights, disc_weights) - prob_real_true(disc_weights)
    return cost


def gen_cost(disc_weights, gen_weights):
    return -prob_fake_true(gen_weights, disc_weights)

Training Initialization:

In [58]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 2) + \
                   np.random.normal(scale=eps, size=(3,))
init_disc_weights = np.random.normal(size=(3,))

gen_weights_grad  = torch.tensor(init_gen_weights, requires_grad=True)
disc_weights_grad = torch.tensor(init_disc_weights, requires_grad=True)

Running training

In [78]:
N = 100     #Number of training cycles
steps = 1 #Number of steps for generator training
ratio = 5  #gen/disc training ratio

for i in range (N):
    
    #training discriminator
    gen_weights_non_grad = torch.tensor(gen_weights_grad.detach().numpy(), requires_grad = False)
    opt = optim.OAdam([disc_weights_grad, gen_weights_non_grad], lr = 0.1)
    
    def closure():
        opt.zero_grad()
        
        loss = disc_cost(disc_weights_grad, gen_weights_non_grad)
        
        disc_weights=np.array( list(disc_weights_grad.detach().numpy()) ,    requires_grad=True )
        gen_weights =np.array( list(gen_weights_non_grad.detach().numpy()) , requires_grad=False)
        
        grad_gen_circuit_disc =qml.grad(gen_disc_circuit_grad,  argnum=1)
        grad_real_disc_circuit=qml.grad(real_disc_circuit_grad, argnum=1) 
        grad_fn_disc= grad_gen_circuit_disc(gen_weights, disc_weights)-grad_real_disc_circuit(angles, disc_weights)

        disc_weights_grad.grad = torch.tensor(grad_fn_disc)

        return loss

    for j in range(steps*ratio):
        opt.step(closure)
        if (j+1)%5==0:
            print('step : ',j, ' discriminator cost : ',disc_cost(disc_weights_grad, gen_weights_non_grad).item())
    print('discriminator trained')
    
    #training generator
    disc_weights_non_grad = torch.tensor(disc_weights_grad.detach().numpy(), requires_grad = False)
    opt = optim.OAdam([disc_weights_non_grad, gen_weights_grad], lr = 0.1)

    def closure():
        opt.zero_grad()
        loss = gen_cost(disc_weights_non_grad, gen_weights_grad)
        
        disc_weights=np.array( list(disc_weights_grad.detach().numpy()) ,    requires_grad=False)
        gen_weights =np.array( list(gen_weights_non_grad.detach().numpy()) , requires_grad=True)
        
        grad_fn_gen=-qml.grad(gen_disc_circuit_grad, argnum=0)(gen_weights, disc_weights)
        
        gen_weights_grad.grad = torch.tensor(grad_fn_gen)

        return loss

    for j in range(steps):
        opt.step(closure)
        if (j+1)%5==0:
            print('step : ',j, ' generator cost : ',gen_cost(disc_weights_non_grad, gen_weights_grad).item())
        
    print('end of cycle number : ', i+1)


step :  4  discriminator cost :  0.01766390701000964
discriminator trained
end of cycle number :  1
step :  4  discriminator cost :  0.08544712738671306
discriminator trained
end of cycle number :  2
step :  4  discriminator cost :  -0.0469636514739753
discriminator trained
end of cycle number :  3
step :  4  discriminator cost :  -0.229681082688741
discriminator trained
end of cycle number :  4
step :  4  discriminator cost :  -0.2197763329951518
discriminator trained
end of cycle number :  5
step :  4  discriminator cost :  -0.17505745723383537
discriminator trained
end of cycle number :  6
step :  4  discriminator cost :  -0.04874061381585815
discriminator trained
end of cycle number :  7
step :  4  discriminator cost :  0.039632097177686454
discriminator trained
end of cycle number :  8
step :  4  discriminator cost :  -0.012461646125006554
discriminator trained
end of cycle number :  9
step :  4  discriminator cost :  -0.03687464720804634
discriminator trained
end of cycle number 

step :  4  discriminator cost :  -0.023110610297288448
discriminator trained
end of cycle number :  83
step :  4  discriminator cost :  -0.04658620681119996
discriminator trained
end of cycle number :  84
step :  4  discriminator cost :  -0.023073111521679457
discriminator trained
end of cycle number :  85
step :  4  discriminator cost :  -0.04668863052376515
discriminator trained
end of cycle number :  86
step :  4  discriminator cost :  -0.02303758055628513
discriminator trained
end of cycle number :  87
step :  4  discriminator cost :  -0.04678659106681127
discriminator trained
end of cycle number :  88
step :  4  discriminator cost :  -0.023003943730407705
discriminator trained
end of cycle number :  89
step :  4  discriminator cost :  -0.04688026896310471
discriminator trained
end of cycle number :  90
step :  4  discriminator cost :  -0.022972129899007987
discriminator trained
end of cycle number :  91
step :  4  discriminator cost :  -0.04696983702760704
discriminator trained
en

In [79]:
print("Prob(real classified as real): ", prob_real_true(disc_weights_grad).item())

Prob(real classified as real):  0.939634237336783


In [80]:
print("Prob(fake classified as real): ", prob_fake_true(gen_weights_grad, disc_weights_grad).item())


Prob(fake classified as real):  0.9597010098586476


Comparing output for |0> and printing tthe rotation params of the generator:

In [81]:
obs = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]

bloch_vector_real = qml.map(real, obs, dev, interface="tf")
bloch_vector_generator = qml.map(generator, obs, dev, interface="tf")

print("Real Bloch vector: {}".format(bloch_vector_real([phi, theta, omega])))
print("Generator Bloch vector: {}".format(bloch_vector_generator(gen_weights_grad.detach().numpy())))


Real Bloch vector: [0.90096887 0.43388374 0.        ]
Generator Bloch vector: [0.9288218  0.36498883 0.06382172]
